## 6. Agent : PropositionalLogicAgent (Definitions)

Cet agent est specialise dans l'analyse logique formelle (logique propositionnelle) en utilisant la bibliotheque Java Tweety via JPype.

**Role :**
*   Traduire des extraits de texte/arguments en un "Belief Set" PL (format Tweety) (`PLAnalyzer.semantic_TextToPLBeliefSet`, puis `StateManager.add_belief_set`).
*   Generer des requetes logiques pertinentes pour interroger un Belief Set (`PLAnalyzer.semantic_GeneratePLQueries`).
*   Executer ces requetes via la fonction native interfacant avec Tweety (`PLAnalyzer.execute_and_log_pl_query` - **RECOMMANDE** car logging automatique, ou `PLAnalyzer.execute_pl_query` + `StateManager.log_query_result`). **Necessite une JVM prete.**
*   Interpreter les resultats bruts des requetes en langage naturel (`PLAnalyzer.semantic_InterpretPLResult`).
*   Repondre aux taches assignees par le PM (`StateManager.add_answer`).

**Composants Definis Ci-dessous :**
*   `PropositionalLogicPlugin` (Classe V11 - avec logging automatique)
*   Prompts Semantiques (`prompt_text_to_pl_v8`, `prompt_gen_pl_queries_v8`, `prompt_interpret_pl_v8`) et Fonction Setup (`setup_pl_kernel`)
*   Instructions Systeme (`PL_AGENT_INSTRUCTIONS` - V12)

**Syntaxe Cruciale :** Le Belief Set et les requetes doivent respecter la BNF du `PlParser` de Tweety. L'operateur `>>` est a eviter.

**Nouveaute V11:** La fonction `execute_and_log_pl_query` combine execution ET logging pour garantir que les resultats apparaissent dans la cross-validation finale.

### 🔌 Classe Plugin : PropositionalLogicPlugin

In [ ]:
# %% CELLULE [6.1] - Classe PropositionalLogicPlugin
# (Remplace une partie de l'ancienne cellule 7bbc31e5)
# V11 - Ajout execute_and_log_pl_query pour logging automatique des résultats

import logging
import jpype
from typing import List, Optional, Any, Dict, Callable
from semantic_kernel.functions import kernel_function
import time
from datetime import datetime

logger = logging.getLogger("Orchestration.AgentPL.Defs")
plugin_logger = logging.getLogger("Orchestration.PLAnalyzerPlugin")

# --- Configuration Logger Plugin ---
if not plugin_logger.handlers and not plugin_logger.propagate:
     handler = logging.StreamHandler(); formatter = logging.Formatter('%(asctime)s [%(levelname)s] [%(name)s] %(message)s', datefmt='%H:%M:%S'); handler.setFormatter(formatter); plugin_logger.addHandler(handler); plugin_logger.setLevel(logging.INFO)

# --- Plugin Spécifique PLAnalyzer (V11 - avec logging automatique) ---
class PropositionalLogicPlugin:
    """
    Plugin SK pour l'analyse en logique propositionnelle via Tweety/JPype.
    Contient la logique d'interaction Java et expose une fonction native au kernel.
    V11: Ajout de execute_and_log_pl_query pour logging automatique.
    """
    _logger: logging.Logger
    _jvm_ok: bool
    _PlParser: Optional[Any] = None
    _SatReasoner: Optional[Any] = None
    _PlFormula: Optional[Any] = None
    _parser_instance: Optional[Any] = None
    _reasoner_instance: Optional[Any] = None
    _log_callback: Optional[Callable] = None  # Callback vers StateManager.log_query_result
    _query_log: List[Dict] = None  # Log interne des requêtes pour cross-validation

    def __init__(self, log_callback: Optional[Callable] = None):
        self._logger = plugin_logger
        self._jvm_ok = False
        self._log_callback = log_callback
        self._query_log = []  # Initialiser le log interne
        self._logger.info("Instance PropositionalLogicPlugin créée (V11 avec logging automatique).")
        self._initialize_jvm_components()

    def set_log_callback(self, callback: Callable):
        """Configure le callback pour logger automatiquement les résultats de requêtes."""
        self._log_callback = callback
        self._logger.info("Callback de logging configuré pour PropositionalLogicPlugin.")

    def get_query_log(self) -> List[Dict]:
        """Retourne le log interne des requêtes exécutées."""
        return self._query_log.copy()

    def _initialize_jvm_components(self):
        """Vérifie la JVM et charge les classes/instances Tweety."""
        global jvm_ready # Vérifier la variable globale définie dans la cellule JPype
        if 'jvm_ready' in globals() and jvm_ready and 'jpype' in globals() and jpype.isJVMStarted():
            self._logger.info("JVM détectée comme prête. Chargement classes Tweety...")
            self._jvm_ok = True
        else:
            self._logger.critical("JVM non prête lors init Plugin PL! Fonctions natives échoueront.")
            self._jvm_ok = False
            return

        try:
            self._PlParser = jpype.JClass("org.tweetyproject.logics.pl.parser.PlParser")
            self._SatReasoner = jpype.JClass("org.tweetyproject.logics.pl.reasoner.SatReasoner")
            self._PlFormula = jpype.JClass("org.tweetyproject.logics.pl.syntax.PlFormula")
            self._parser_instance = self._PlParser()
            self._reasoner_instance = self._SatReasoner()
            self._logger.info("Classes et instances Java Tweety chargées.")
        except Exception as e:
            self._logger.critical(f"Erreur chargement classes/instances Tweety: {e}", exc_info=True)
            self._jvm_ok = False

    # --- Méthodes Internes ---
    def _internal_parse_formula(self, formula_string: str) -> Optional[Any]:
        if not self._jvm_ok or not self._parser_instance:
            self._logger.error(f"Parse formula: JVM/Parser non prêt ('{formula_string[:60]}...').")
            return None
        try:
            self._logger.debug(f"Parsing formule: '{formula_string}'")
            return self._parser_instance.parseFormula(formula_string)
        except jpype.JException as e_java:
            error_msg = f"Erreur Java parsing formule '{formula_string}': {e_java.getClass().getName()}: {e_java.getMessage()}"
            self._logger.error(error_msg)
            raise RuntimeError(f"Erreur Parsing Formule: {e_java.getMessage()}") from e_java
        except Exception as e:
            self._logger.error(f"Erreur Python parsing formule '{formula_string}': {e}", exc_info=True)
            raise RuntimeError(f"Erreur Python Parsing Formule: {e}") from e

    def _internal_parse_belief_set(self, belief_set_string: str) -> Optional[Any]:
        if not self._jvm_ok or not self._parser_instance:
            self._logger.error(f"Parse BS: JVM/Parser non prêt (BS: '{belief_set_string[:60]}...').")
            return None
        try:
            belief_set_string_cleaned = belief_set_string.replace("\\\\n", "\\n")
            self._logger.debug(f"Parsing belief set (nettoyé): '{belief_set_string_cleaned[:100]}...'")
            parsed_bs = self._parser_instance.parseBeliefBase(belief_set_string_cleaned)
            self._logger.debug(f" -> BS parsé (type: {type(parsed_bs)}, taille: {parsed_bs.size()}).")
            return parsed_bs
        except jpype.JException as e_java:
            error_msg = f"Erreur Java parsing BS (extrait: '{belief_set_string[:60]}...'): {e_java.getClass().getName()}: {e_java.getMessage()}"
            self._logger.error(error_msg)
            raise RuntimeError(f"Erreur Parsing Belief Set: {e_java.getMessage()}") from e_java
        except Exception as e:
            self._logger.error(f"Erreur Python parsing BS: {e}", exc_info=True)
            raise RuntimeError(f"Erreur Python Parsing Belief Set: {e}") from e

    def _internal_execute_query(self, belief_set_obj: Any, formula_obj: Any) -> Optional[bool]:
        if not self._jvm_ok or not self._reasoner_instance or not self._PlFormula:
            self._logger.error("Execute query: JVM/Reasoner/Formula non prêt.")
            return None
        try:
            if not isinstance(formula_obj, self._PlFormula):
                 raise TypeError(f"Objet requête n'est pas un PlFormula (type: {type(formula_obj)})")
            self._logger.debug(f"Exécution requête avec raisonneur '{self._reasoner_instance.getClass().getName()}'...")
            result_java_boolean = self._reasoner_instance.query(belief_set_obj, formula_obj)
            self._logger.debug(f" -> Résultat brut Java: {result_java_boolean}")
            if result_java_boolean is None:
                self._logger.warning("Requête Tweety a retourné null (indéterminé?).")
                return None
            else:
                result_python_bool = bool(result_java_boolean)
                self._logger.info(f" -> Résultat requête Python: {result_python_bool}")
                return result_python_bool
        except jpype.JException as e_java:
            error_msg = f"Erreur Java exécution requête: {e_java.getClass().getName()}: {e_java.getMessage()}"
            self._logger.error(error_msg)
            raise RuntimeError(f"Erreur Exécution Requête Tweety: {e_java.getMessage()}") from e_java
        except Exception as e:
            self._logger.error(f"Erreur Python exécution requête: {e}", exc_info=True)
            raise RuntimeError(f"Erreur Python Exécution Requête: {e}") from e

    # --- Méthode Façade (ancienne, conservée pour compatibilité) ---
    @kernel_function(
        description="Exécute une requête en Logique Propositionnelle (syntaxe Tweety: !,||,=>,<=>,^^) sur un Belief Set fourni. Retourne le résultat (ACCEPTED, REJECTED, Unknown, ou FUNC_ERROR). NOTE: Préférez execute_and_log_pl_query qui enregistre automatiquement le résultat.",
        name="execute_pl_query"
    )
    def execute_pl_query(self, belief_set_content: str, query_string: str) -> str:
        self._logger.info(f"Appel execute_pl_query: Query='{query_string}' sur BS ('{belief_set_content[:60]}...')")
        if not self._jvm_ok:
            self._logger.error("execute_pl_query: JVM non prête.")
            return "FUNC_ERROR: JVM non prête ou composants Tweety non chargés."
        try:
            belief_set_obj = self._internal_parse_belief_set(belief_set_content)
            if belief_set_obj is None: return "FUNC_ERROR: Échec parsing Belief Set. Vérifiez syntaxe."
            formula_obj = self._internal_parse_formula(query_string)
            if formula_obj is None: return f"FUNC_ERROR: Échec parsing requête '{query_string}'. Vérifiez syntaxe."
            result_bool: Optional[bool] = self._internal_execute_query(belief_set_obj, formula_obj)
            if result_bool is None:
                result_str = f"Tweety Result: Unknown for query '{query_string}'."
                self._logger.warning(f"Requête '{query_string}' -> indéterminé (None).")
            else:
                result_label = "ACCEPTED (True)" if result_bool else "REJECTED (False)"
                result_str = f"Tweety Result: Query '{query_string}' is {result_label}."
                self._logger.info(f" -> Résultat formaté requête '{query_string}': {result_label}")
            return result_str
        except RuntimeError as e_runtime:
             self._logger.error(f"Erreur exécution (parsing/query): {e_runtime}")
             return f"FUNC_ERROR: {e_runtime}"
        except Exception as e_py:
            error_msg = f"Erreur Python inattendue dans execute_pl_query: {e_py}"
            self._logger.error(error_msg, exc_info=True)
            return f"FUNC_ERROR: {error_msg}"

    # --- Nouvelle Méthode Fusionnée (V11) ---
    @kernel_function(
        description="Exécute une requête PL ET enregistre automatiquement le résultat dans l'état. RECOMMANDÉE pour toutes les requêtes PL car elle garantit le logging pour la cross-validation.",
        name="execute_and_log_pl_query"
    )
    def execute_and_log_pl_query(self, belief_set_id: str, belief_set_content: str, query_string: str) -> str:
        """
        Exécute une requête PL et enregistre automatiquement le résultat.
        
        Args:
            belief_set_id: ID du belief set (ex: "propositional_bs_1")
            belief_set_content: Contenu du belief set en syntaxe Tweety
            query_string: Requête PL à exécuter
            
        Returns:
            Résultat formaté incluant confirmation du logging
        """
        self._logger.info(f"Appel execute_and_log_pl_query: Query='{query_string}' sur BS '{belief_set_id}'")
        
        # 1. Exécuter la requête
        result_str = self.execute_pl_query(belief_set_content, query_string)
        
        # 2. Parser le résultat pour extraire le statut
        if "ACCEPTED (True)" in result_str:
            status = "ACCEPTED"
        elif "REJECTED (False)" in result_str:
            status = "REJECTED"
        elif "Unknown" in result_str:
            status = "UNKNOWN"
        elif "FUNC_ERROR" in result_str:
            status = "ERROR"
        else:
            status = "UNKNOWN"
        
        # 3. Créer l'entrée de log
        log_entry = {
            "belief_set_id": belief_set_id,
            "query": query_string,
            "result": status,
            "raw_result": result_str,
            "timestamp": datetime.now().isoformat()
        }
        
        # 4. Logger en interne (toujours)
        self._query_log.append(log_entry)
        self._logger.info(f"Requête loggée en interne: {query_string} -> {status}")
        
        # 5. Appeler le callback externe si disponible (StateManager.log_query_result)
        callback_status = "non configuré"
        if self._log_callback is not None:
            try:
                # Le callback attend: belief_set_id, query_string, result, details (optionnel)
                callback_result = self._log_callback(
                    belief_set_id=belief_set_id,
                    query_string=query_string,
                    result=status,
                    details=result_str
                )
                callback_status = "OK"
                self._logger.info(f"Callback logging externe appelé avec succès.")
            except Exception as e:
                callback_status = f"ERREUR: {e}"
                self._logger.error(f"Erreur appel callback logging: {e}")
        
        # 6. Retourner résultat enrichi
        return f"{result_str}\n[LOGGED: BS={belief_set_id}, Status={status}, Callback={callback_status}]"


logger.info("Classe PropositionalLogicPlugin (V11 avec logging automatique) définie.")

### 📜 Prompts Sémantiques et ⚙️ Fonction Setup (PL)

In [ ]:
# %% CELLULE [6.2] - Prompts Semantiques et Fonction Setup (PL) - V11
# (Remplace une partie de l'ancienne cellule 7bbc31e5 et corrige l'indentation)
# V11 - Support du callback logging pour execute_and_log_pl_query

import semantic_kernel as sk
from semantic_kernel.functions import kernel_function
import logging

# S'assurer que les dependances sont la
if 'PropositionalLogicPlugin' not in globals(): raise NameError("Classe PropositionalLogicPlugin non definie.")

logger = logging.getLogger("Orchestration.AgentPL.Setup")

# --- Fonctions Semantiques PLAnalyzer (Prompts V8 avec BNF) ---
prompt_text_to_pl_v8 = """
[Instructions]
Transformez le texte fourni ($input) en un belief set PL.
Respectez **STRICTEMENT** la BNF Tweety ci-dessous. Retournez les formules une par ligne, separees par '\\n'.
N'utilisez **PAS** '&&' ou '>>'. Utilisez les operateurs !, ||, =>, <=>, ^^.
Utilisez des noms de propositions courts et significatifs (ex: `renewable_essential`, `high_cost`).

```bnf
FORMULASET ::== FORMULA ( "\\n" FORMULA )*
FORMULA ::== PROPOSITION | "(" FORMULA ")" | FORMULA ">>" FORMULA |
             FORMULA "||" FORMULA | FORMULA "=&lt;" FORMULA | FORMULA "&gt;=&lt;" FORMULA |
             FORMULA "^^" FORMULA | "!" FORMULA | "+" | "-"
PROPOSITION is a sequence of characters excluding |,&,!,(),=,<,> and whitespace.
(Rappel: N'utilisez PAS '>>', utilisez !, ||, =>, <=> si besoin).

[Texte a Analyser]
{{$input}}
+++++
[Belief Set PL (Syntaxe Tweety Valide, une formule par ligne)]
"""

prompt_gen_pl_queries_v8 = """
[Instructions]
Etant donne un texte original ($input) et un belief set PL ($belief_set), generez 2-3 requetes PL pertinentes pour verifier la coherence ou deduire des informations.
Respectez STRICTEMENT la BNF PL Tweety ci-dessous. Retournez les requetes une par ligne.
N'utilisez PAS '&&' ou '>>'. Utilisez les operateurs !, ||, =>, <=>, ^^.
Assurez-vous que les propositions utilisees dans les requetes existent dans le belief set ou sont des combinaisons logiques de celles-ci.
FORMULA ::== PROPOSITION | "(" FORMULA ")" | FORMULA ">>" FORMULA |
             FORMULA "||" FORMULA | FORMULA "=&lt;" FORMULA | FORMULA "&gt;=&lt;" FORMULA |
             FORMULA "^^" FORMULA | "!" FORMULA | "+" | "-"
PROPOSITION is a sequence of characters excluding |,&,!,(),=,<,> and whitespace.
(Rappel: N'utilisez PAS '>>').

[Texte Original]
{{$input}}
[Belief Set PL]
{{$belief_set}}
+++++
[Requetes PL Generees (Syntaxe Tweety Valide, une par ligne)]
"""

prompt_interpret_pl_v8 = """
[Instructions]
Interpretez en langage naturel clair le resultat Tweety formate ($tweety_result) pour une ou plusieurs requetes PL ($queries).
Le resultat Tweety pour chaque requete indique si elle est 'ACCEPTED (True)' ou 'REJECTED (False)' ou 'Unknown'.
Basez votre interpretation sur le belief set PL fourni ($belief_set) et le texte original ($input).

Pour chaque requete :

     Rappelez la requete.
     Indiquez si elle est Acceptee, Rejetee ou Inconnue.
     Expliquez pourquoi en vous referant aux formules specifiques du belief set qui justifient le resultat.
         Si ACCEPTED: Montrez comment la requete decoule logiquement des formules du belief set.
         Si REJECTED: Expliquez que la requete N'EST PAS une consequence logique du belief set (elle n'est pas prouvable), ce qui ne signifie pas forcement qu'elle est fausse dans l'absolu. Mentionnez si elle contredit des formules.
         Si Unknown: Indiquez que le raisonneur n'a pas pu determiner.

     Reliez l'interpretation au sens du texte original si pertinent.


Generez une interpretation globale concise et facile a comprendre.

[Texte Original]
{{$input}}
[Belief Set PL]
{{$belief_set}}
[Requetes Testees (une par ligne)]
{{$queries}}
[Resultats Formates Tweety Agreges (un resultat par ligne)]
{{$tweety_result}}
+++++
[Interpretation Detaillee en Langage Naturel]
"""
logger.debug("Prompts semantiques PL (V8) definis.")

# --- Fonction pour configurer le Kernel specifique a l'agent PL (V11 - Callback Logging) ---
def setup_pl_kernel(kernel: sk.Kernel, llm_service, state_manager_plugin=None):
    """
    Configure le kernel pour le PropositionalLogicAgent.
    Ajoute une instance du PropositionalLogicPlugin et les fonctions semantiques.
    
    Args:
        kernel: Kernel Semantic Kernel a configurer
        llm_service: Service LLM pour les fonctions semantiques
        state_manager_plugin: Instance optionnelle du StateManagerPlugin pour configurer le callback logging
    """
    plugin_name = "PLAnalyzer"
    logger.info(f"Configuration Kernel pour {plugin_name} (V11 - Callback Logging)...")

    # Configurer le callback de logging si state_manager_plugin est fourni
    log_callback = None
    if state_manager_plugin is not None:
        if hasattr(state_manager_plugin, 'log_query_result'):
            log_callback = state_manager_plugin.log_query_result
            logger.info("Callback log_query_result configure depuis StateManagerPlugin.")
        else:
            logger.warning("StateManagerPlugin fourni mais sans methode log_query_result.")

    # Instanciation du plugin avec le callback
    pl_plugin_instance = PropositionalLogicPlugin(log_callback=log_callback)

    # Ajout du plugin au kernel passe en argument
    if plugin_name in kernel.plugins: logger.warning(f"Plugin '{plugin_name}' deja present. Remplacement.")
    kernel.add_plugin(pl_plugin_instance, plugin_name=plugin_name)
    logger.debug(f"Instance du plugin '{plugin_name}' ajoutee/mise a jour.")

    # Configuration des settings LLM
    default_settings = None
    if llm_service:
        try: default_settings = kernel.get_prompt_execution_settings_from_service_id(llm_service.service_id); logger.debug(f"Settings LLM recuperes pour {plugin_name}.")
        except Exception as e: logger.warning(f"Impossible de recuperer settings LLM pour {plugin_name}: {e}")

    # Ajout des fonctions semantiques au kernel
    semantic_functions = [
        ("semantic_TextToPLBeliefSet", prompt_text_to_pl_v8, "Traduit texte en Belief Set PL (syntaxe Tweety ! || =&lt; &gt;=&lt; ^^)."),
        ("semantic_GeneratePLQueries", prompt_gen_pl_queries_v8, "Genere requetes PL pertinentes (syntaxe Tweety ! || =&lt; &gt;=&lt; ^^)."),
        ("semantic_InterpretPLResult", prompt_interpret_pl_v8, "Interprete resultat requete PL Tweety formate.")
    ]
    for func_name, prompt, description in semantic_functions:
        try:
            kernel.add_function(prompt=prompt, plugin_name=plugin_name, function_name=func_name, description=description, prompt_execution_settings=default_settings)
            logger.debug(f"Fonction semantique {plugin_name}.{func_name} ajoutee/mise a jour.")
        except ValueError as ve: logger.warning(f"Probleme ajout/MaJ {plugin_name}.{func_name}: {ve}")

    # Verification des fonctions natives (facade + nouvelle fusionnee)
    native_functions = ["execute_pl_query", "execute_and_log_pl_query"]
    if plugin_name in kernel.plugins:
        for native_func in native_functions:
            if native_func not in kernel.plugins[plugin_name]:
                logger.error(f"ERREUR: Fonction native {plugin_name}.{native_func} non enregistree!")
            else:
                logger.debug(f"Fonction native {plugin_name}.{native_func} trouvee.")
    else:
        logger.error(f"ERREUR CRITIQUE: Plugin {plugin_name} non trouve apres ajout!")

    logger.info(f"Kernel {plugin_name} configure (V11 avec callback logging).")
    
    # Retourner l'instance du plugin pour permettre l'acces au query_log
    return pl_plugin_instance

# Fin de la definition de la fonction setup_pl_kernel

### 📜 Instructions Système : PL_AGENT_INSTRUCTIONS

In [ ]:
# %% CELLULE [6.3] - Instructions Systeme (PL)
# V14 - CRITIQUE: add_answer AVANT tout texte

import logging

logger = logging.getLogger("Orchestration.AgentPL.Instructions")

# --- Instructions Systeme PL Agent (V14 - add_answer AVANT texte) ---
PL_AGENT_INSTRUCTIONS_V14 = """
Votre Role: Specialiste en logique propositionnelle (Tweety).

**REGLES CRITIQUES D'EXECUTION:**

1. **NE JAMAIS ECRIRE DE TEXTE LIBRE** - Ecrire du texte termine votre tour SANS appeler add_answer!
2. **UNIQUEMENT DES APPELS DE FONCTION** - Tout votre travail passe par StateManager et PLAnalyzer.
3. **add_answer EST OBLIGATOIRE** - C'est TOUJOURS votre DERNIER appel de fonction.
4. **INTERDIT: set_final_conclusion** - Reserve au ProjectManagerAgent.

**Syntaxe Tweety:** N'utilisez PAS >>. Utilisez !, ||, =>, <=>, ^^.

**Fonctions Disponibles:**
- `StateManager.get_current_state_snapshot(summarize)`: Lire l'etat.
- `StateManager.add_belief_set(logic_type, content)`: Enregistrer un belief set.
- `StateManager.add_answer(task_id, author_agent, answer_text, source_ids)`: **OBLIGATOIRE - FIN DE TACHE**
- `PLAnalyzer.semantic_TextToPLBeliefSet(input)`: Traduire en PL.
- `PLAnalyzer.semantic_GeneratePLQueries(input, belief_set)`: Generer requetes.
- `PLAnalyzer.execute_and_log_pl_query(belief_set_id, belief_set_content, query_string)`: **RECOMMANDE** - Execute ET logue.
- `PLAnalyzer.semantic_InterpretPLResult(...)`: Interpreter resultats.

**WORKFLOW OBLIGATOIRE - TRADUCTION PL:**

```
ETAPE 1: Lire l'etat
  -> StateManager.get_current_state_snapshot(summarize=False)

ETAPE 2: Traduire en PL
  -> PLAnalyzer.semantic_TextToPLBeliefSet(input="[texte arguments]")

ETAPE 3: Enregistrer le belief set
  -> StateManager.add_belief_set(logic_type="Propositional", content="[formules]")

ETAPE 4: OBLIGATOIRE - Signaler fin
  -> StateManager.add_answer(
       task_id="task_X",
       author_agent="PropositionalLogicAgent",
       answer_text="Belief set cree avec N formules.",
       source_ids=["propositional_bs_1"]
     )

ETAPE 5: STOP - NE RIEN FAIRE DE PLUS (pas de texte!)
```

**WORKFLOW OBLIGATOIRE - REQUETES PL:**

```
ETAPE 1: Lire l'etat (recuperer belief_set_id et content)
  -> StateManager.get_current_state_snapshot(summarize=False)

ETAPE 2: Generer requetes
  -> PLAnalyzer.semantic_GeneratePLQueries(input, belief_set)

ETAPE 3: Executer 2-3 requetes MAX (avec logging auto)
  -> PLAnalyzer.execute_and_log_pl_query(bs_id, bs_content, query1)
  -> PLAnalyzer.execute_and_log_pl_query(bs_id, bs_content, query2)

ETAPE 4: OBLIGATOIRE - Signaler fin
  -> StateManager.add_answer(
       task_id="task_X",
       author_agent="PropositionalLogicAgent",
       answer_text="N requetes executees.",
       source_ids=[]
     )

ETAPE 5: STOP - NE RIEN FAIRE DE PLUS (pas de texte!)
```

**ERREURS A EVITER:**
- Ecrire "Voici les resultats..." -> TERMINE LE TOUR SANS add_answer!
- Executer trop de requetes (>3) -> Risque de timeout!
- Oublier add_answer -> PM bloque en attente indefiniment!

**RAPPEL FINAL:**
Votre tour se termine des que vous ecrivez du texte.
Appelez add_answer AVANT tout texte, ou mieux: N'ECRIVEZ PAS DE TEXTE DU TOUT.
""" 

PL_AGENT_INSTRUCTIONS = PL_AGENT_INSTRUCTIONS_V14

logger.info("Instructions Systeme PL_AGENT_INSTRUCTIONS (V14 - add_answer AVANT texte) definies.")

## 6.1. Test du Plugin PropositionalLogicPlugin (Tweety) - Commenté

Cette cellule, **commentée par défaut**, contient du code pour tester isolément le `PropositionalLogicPlugin`, en particulier son interaction avec Tweety via JPype.

**Objectif du test (s'il était activé) :**
*   Vérifier que la JVM est prête.
*   Instancier le plugin.
*   Exécuter la fonction native `execute_pl_query` avec un belief set et des requêtes prédéfinis.
*   Analyser les résultats (ACCEPTED/REJECTED/Error).

**Prérequis INDISPENSABLES (si activé) :**
1.  La **JVM doit être démarrée correctement** (voir cellule 2). La variable `jvm_ready` doit être `True`.
2.  La cellule de définition du `PropositionalLogicPlugin` doit avoir été exécutée.

**Statut actuel :** Laissé commenté pour se concentrer sur le flux principal. Peut être utile pour déboguer l'interaction JPype/Tweety si l'agent PL rencontre des problèmes.

In [ ]:
# # %% Test du plugin PropositionalLogicPlugin (V6 - Contradiction Explicite)
# import logging
# import jpype # Importer pour vérifier

# test_pl_logger = logging.getLogger("Orchestration.Test.PLPlugin")
# test_pl_logger.setLevel(logging.INFO)
# if not test_pl_logger.handlers:
#     handler = logging.StreamHandler(); formatter = logging.Formatter('%(asctime)s [%(levelname)s] [%(name)s] %(message)s', datefmt='%H:%M:%S'); handler.setFormatter(formatter); test_pl_logger.addHandler(handler)

# test_pl_logger.info("--- Début Test PropositionalLogicPlugin (V6 - Contradiction Explicite) ---")

# # Vérifier prérequis
# jvm_ok_for_test = False
# if 'jvm_ready' in globals() and jvm_ready and 'jpype' in globals() and jpype.isJVMStarted():
#     test_pl_logger.info("✅ Prérequis JVM OK.")
#     jvm_ok_for_test = True
# else:
#     test_pl_logger.error("❌ Prérequis JVM NON rempli. Le test natif échouera probablement.")
#     print("❌ ERREUR: JVM non prête. Exécutez la cellule de configuration JPype.")

# if 'PropositionalLogicPlugin' not in globals():
#      test_pl_logger.error("❌ Classe PropositionalLogicPlugin non trouvée.")
#      print("❌ ERREUR: Classe PropositionalLogicPlugin non définie.")
#      jvm_ok_for_test = False # Empêcher la suite

# if jvm_ok_for_test:
#     try:
#         # 1. Instanciation
#         test_pl_logger.info("1. Instanciation de PropositionalLogicPlugin...")
#         plugin_instance_pl_test = PropositionalLogicPlugin()
#         test_pl_logger.info("   Instance créée.")

#         # 2. Définition données de test (CORRIGÉ V6: Ajout 'imported_fossils' pour rendre la contradiction claire)
#         test_belief_set = """essential_law
# !high_cost || !false_debate
# false_debate || long_term_cost_inaction
# high_cost
# renewable_creates_jobs
# imported_fossils
# !imported_fossils || dependency
# # La ligne '!renewable_creates_jobs || !dependency' est enlevée car elle contredit directement les faits 5 et 6/7
# irresponsible_ignore
# !essential_law || !long_term_cost_inaction || !renewable_creates_jobs || !dependency || !irresponsible_ignore
# """
#         # Les requêtes restent les mêmes
#         test_query_1 = "essential_law" # Attendu: ACCEPTED
#         test_query_2 = "long_term_cost_inaction" # Attendu: ACCEPTED
#         test_query_3 = "cheap_law" # Attendu: REJECTED/Unknown
#         test_query_4 = "!irresponsible_ignore" # Attendu: ACCEPTED (car KB contradictoire)
#         test_query_5 = "non_existent" # Attendu: REJECTED/Unknown
#         test_query_6 = "essential_law || !essential_law" # Tautologie, Attendu: ACCEPTED
#         test_query_7 = "essential_law && !essential_law" # Contradiction, Attendu: REJECTED (si KB était cohérent), ACCEPTED (si KB contradictoire)

#         print("\n--- Données de Test (Contradiction Explicite) ---")
#         print(f"Belief Set:\n{test_belief_set}")
#         queries_to_test = [test_query_1, test_query_2, test_query_3, test_query_4, test_query_5, test_query_6, test_query_7]
#         for i, q in enumerate(queries_to_test): print(f"Requête {i+1}: {q}")


#         # 3. Exécution des requêtes
#         test_pl_logger.info("\n2. Test: native_ExecutePLQuery sur Belief Set Complet V6...")

#         print("\n--- Résultats Tests Requêtes ---")
#         all_queries_passed = True # Optimiste
#         results_summary = []

#         for i, query in enumerate(queries_to_test):
#             test_pl_logger.info(f"   Exécution requête {i+1}: '{query}'")
#             result = plugin_instance_pl_test.native_ExecutePLQuery(test_belief_set, query)
#             test_pl_logger.info(f"   Résultat brut requête {i+1}: {result}")
#             print(f"Résultat Requête {i+1} ('{query}'): {result}")
#             results_summary.append(result)

#             # Vérification (ATTENTION: si KB contradictoire, tout devrait être ACCEPTED sauf erreur)
#             is_error = "FUNC_ERROR:" in result
#             is_accepted = "ACCEPTED (True)" in result
#             is_rejected = "REJECTED (False)" in result or "Unknown" in result # Grouper REJECTED et Unknown

#             if is_error:
#                 test_pl_logger.error(f"   !!! Erreur détectée pour la requête {i+1} : {result} !!!")
#                 all_queries_passed = False
#             elif i == 2 or i == 4 : # cheap_law (Query 3), non_existent (Query 5) -> Devrait être ACCEPTED si KB contradictoire
#                  if not is_accepted: all_queries_passed = False; test_pl_logger.warning(f"Req {i+1} ('{query}') résultat inattendu (devrait être ACCEPTED car KB contradictoire)")
#                  else: test_pl_logger.info(f"   Req {i+1} ('{query}') semble OK (ACCEPTED comme attendu pour KB contradictoire).")
#             # else: # Toutes les autres requêtes (1, 2, 4, 6, 7) devraient être ACCEPTED si KB contradictoire
#             #      if not is_accepted: all_queries_passed = False; test_pl_logger.warning(f"Req {i+1} ('{query}') résultat inattendu (devrait être ACCEPTED car KB contradictoire)")
#             #      else: test_pl_logger.info(f"   Req {i+1} ('{query}') semble OK (ACCEPTED comme attendu pour KB contradictoire).")
#             # Simplification: on s'attend à ce que tout soit ACCEPTED
#             else:
#                 if not is_accepted:
#                     all_queries_passed = False; test_pl_logger.warning(f"Req {i+1} ('{query}') résultat inattendu (devrait être ACCEPTED car KB contradictoire, obtenu: {'REJECTED/Unknown' if is_rejected else result})")
#                 else:
#                     test_pl_logger.info(f"   Req {i+1} ('{query}') semble OK (ACCEPTED comme attendu pour KB contradictoire).")


#         print("\n--- Vérification Globale Requêtes (Attente: Tout ACCEPTED car KB contradictoire) ---")
#         if all_queries_passed:
#             test_pl_logger.info("✅✅✅ Tous les tests de requêtes ont retourné ACCEPTED, comportement attendu pour un KB contradictoire.")
#             print("✅✅✅ Tous les tests de requêtes ont retourné ACCEPTED.")
#         else:
#             test_pl_logger.error("❌❌❌ Le comportement face à la contradiction est inattendu. Certaines requêtes n'ont pas retourné ACCEPTED. Vérifiez les logs et le comportement de Tweety SatReasoner.")
#             print("❌❌❌ Comportement inattendu face à la contradiction. Vérifiez les logs.")
#             print("Résultats obtenus :", results_summary)


#     except Exception as e:
#         test_pl_logger.critical(f"Erreur majeure lors du test du plugin PL: {e}", exc_info=True)
#         print(f"\n !!! ERREUR CRITIQUE PENDANT LE TEST PL : {e} !!!")
# else:
#     print("\n--- Test PL Plugin Sauté (Prérequis non remplis) ---")

# test_pl_logger.info("--- Fin Test PropositionalLogicPlugin (V6 - Contradiction Explicite) ---")